In [1]:
# سلول 1: ایمپورت کتابخانه‌ها و تنظیمات پایه

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# تنظیمات نمایشی برای نمودارها
%matplotlib inline
sns.set(style="whitegrid")

print("کتابخانه‌ها با موفقیت وارد شدند.")


کتابخانه‌ها با موفقیت وارد شدند.


In [2]:


# مسیر فایل CSV
file_path = r'C:\Users\HsH-HsH\weighted_split_criteria\notebooks\datasets\customer_churn_data_preprocessed.csv'

# بارگذاری دیتاست
df = pd.read_csv(file_path)

# نام ستون هدف
target_column = 'Churn'

# جدا کردن ویژگی‌ها و ستون هدف
X = df.drop(target_column, axis=1)
y = df[target_column]

# تبدیل ستون‌های غیر عددی به عددی با one-hot encoding (اگر لازم بود)
X = pd.get_dummies(X)

# تبدیل ستون‌های بولی به float64
X = X.astype('float64')

# تبدیل به آرایه numpy
X_array = np.array(X)
y_array = np.array(y)

# نمایش برای اطمینان
print('X shape:', X_array.shape)
print('y shape:', y_array.shape)
print('X dtype:', X_array.dtype)
print('y dtype:', y_array.dtype)


X shape: (896, 7)
y shape: (896,)
X dtype: float64
y dtype: int64


In [3]:
# مسیر کامل فایل CSV
file_path = r'C:\Users\HsH-HsH\weighted_split_criteria\notebooks\datasets\customer_churn_data_preprocessed.csv'

# بارگذاری دیتاست
df = pd.read_csv(file_path)

In [4]:
import sys
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src")
print("پوشه src به sys.path اضافه شد.")

پوشه src به sys.path اضافه شد.


In [5]:
from criteria.gini import gini_criterion
from criteria.qg import gain_ratio_criterion
from criteria.twoing import twoing_criterion
from criteria.ng import normalized_gain_criterion
from criteria.mch import multi_class_hellinger
from criteria.marsh import marsh_criterion
from criteria.gs import g_statistic_criterion
from criteria.dkm import dkm_criterion
from criteria.cs import chi_squared_criterion
from criteria.bhy import bhattacharyya_criterion
from criteria.ks import kolmogorov_smirnov_criterion

criteria_with_weights = [
    ("gini", gini_criterion, 0.15),
    ("gain_ratio", gain_ratio_criterion, 0.15),
    ("twoing", twoing_criterion, 0.10),
    ("normalized_gain", normalized_gain_criterion, 0.10),
    ("multi_class_hellinger", multi_class_hellinger, 0.10),
    ("marshall", marsh_criterion, 0.05),
    ("g_statistic", g_statistic_criterion, 0.05),
    ("dkm", dkm_criterion, 0.10),
    ("chi_squared", chi_squared_criterion, 0.05),
    ("bhattacharyya", bhattacharyya_criterion, 0.10),
    ("kolmogorov_smirnov", kolmogorov_smirnov_criterion, 0.05),
]

print("معیارها و وزن‌ها تعریف شدند.")

معیارها و وزن‌ها تعریف شدند.


In [6]:
import sys
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src\model")
print("پوشه model به sys.path اضافه شد.")

پوشه model به sys.path اضافه شد.


In [7]:
from weighted_decision_tree import WeightedDecisionTreeModel

In [49]:
model = WeightedDecisionTreeModel(criteria_with_weights, positive_label=1, max_depth=15)


In [31]:
model.fit(X_array, y_array)


In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")
print("کتابخانه‌ها با موفقیت وارد شدند.")


کتابخانه‌ها با موفقیت وارد شدند.


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=42)
print('Train shape:', X_train.shape, y_train.shape)
print('Test shape:', X_test.shape, y_test.shape)


Train shape: (716, 7) (716,)
Test shape: (180, 7) (180,)


In [52]:
model.fit(X_train, y_train)


In [53]:
# پیش‌بینی احتمالات کلاس‌ها روی داده تست
probas = model.predict_proba(X_test)

# نمایش ۵ نمونه اول احتمالات پیش‌بینی شده
print(probas[:5])


[[0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [55]:
import numpy as np

# فرض بر این است که proba آرایه ۲ بعدی باشد: (نمونه‌ها، کلاس‌ها)
# برای هر نمونه، کلاس با بیشترین احتمال را انتخاب می‌کنیم
y_pred = np.argmax(probas, axis=1)

# نمایش ۵ نمونه اول پیش‌بینی عددی
print(y_pred[:5])


[1 0 0 1 1]


In [56]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))


Accuracy: 0.7333333333333333
Confusion Matrix:
 [[45 23]
 [25 87]]
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65        68
           1       0.79      0.78      0.78       112

    accuracy                           0.73       180
   macro avg       0.72      0.72      0.72       180
weighted avg       0.73      0.73      0.73       180



In [19]:
import time

# زمان آموزش مدل
start_fit = time.time()
model.fit(X_train, y_train)
fit_time = time.time() - start_fit
print('زمان آموزش مدل (ثانیه):', fit_time)

# زمان پیش‌بینی مدل
start_pred = time.time()
probas = model.predict_proba(X_test)
y_pred = np.argmax(probas, axis=1)
pred_time = time.time() - start_pred
print('زمان پیش‌بینی مدل (ثانیه):', pred_time)


زمان آموزش مدل (ثانیه): 1.980409860610962
زمان پیش‌بینی مدل (ثانیه): 0.003001689910888672


In [42]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# پیش‌بینی احتمالات کلاس‌ها روی داده تست
probas = model.predict_proba(X_test)

# تبدیل احتمالات به پیش‌بینی عددی کلاس
y_pred = np.argmax(probas, axis=1)

# ارزیابی مدل با معیارهای استاندارد
print('دقت کل (Accuracy):', accuracy_score(y_test, y_pred))
print('ماتریس در هم‌ریختگی (Confusion Matrix):\n', confusion_matrix(y_test, y_pred))
print('گزارش طبقه‌بندی (Classification Report):\n', classification_report(y_test, y_pred, digits=4))


دقت کل (Accuracy): 0.7833333333333333
ماتریس در هم‌ریختگی (Confusion Matrix):
 [[ 41  27]
 [ 12 100]]
گزارش طبقه‌بندی (Classification Report):
               precision    recall  f1-score   support

           0     0.7736    0.6029    0.6777        68
           1     0.7874    0.8929    0.8368       112

    accuracy                         0.7833       180
   macro avg     0.7805    0.7479    0.7573       180
weighted avg     0.7822    0.7833    0.7767       180



In [48]:
import time

for depth in [2, 5, 10, 15]:
    model = WeightedDecisionTreeModel(criteria_with_weights, positive_label=1, max_depth=depth)
    start_time = time.time()
    model.fit(X_array, y_array)
    fit_time = time.time() - start_time

    # اگر متد پیش‌بینی مدل مشابه قبلی است:
    probas = model.predict_proba(X_array)
    y_pred = probas.argmax(axis=1)
    
    # اگر y_array همان y_test است (یا باید جدا کنی):
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_array, y_pred)

    print(f"max_depth={depth} | زمان آموزش: {fit_time:.3f}s | دقت: {accuracy:.4f}")


max_depth=2 | زمان آموزش: 0.649s | دقت: 0.8292
max_depth=5 | زمان آموزش: 2.191s | دقت: 0.8605
max_depth=10 | زمان آموزش: 4.902s | دقت: 0.9062
max_depth=15 | زمان آموزش: 6.390s | دقت: 0.9487


In [23]:
# اگر مدل متد print_tree دارد، می‌توانی درخت را چاپ کنی و ببینی چقدر بزرگ است
if hasattr(model.model, 'print_tree'):
    print("ساختار درخت:")
    model.model.print_tree()
else:
    print("متد print_tree وجود ندارد.")


ساختار درخت:
[1] -> metric = 0.0 | repartition = [252, 464]
|    Δ metric = +29.371265103247396
|   [2] feature 3 <= 0.0 -> metric = 0.0 | repartition = [216, 110]
|   |    Δ metric = +7.317069551155959
|   |   [4] feature 0 <= 30.0 -> metric = 0.0 | repartition = [30, 62]
|   |   |    Δ metric = +3.760624649777607
|   |   |   [8] feature 6 <= 0.0 -> metric = 0.0 | repartition = [17, 3]
|   |   |   |    Δ metric = +0.5816535915564661
|   |   |   |   [16] feature 1 <= 148.0 -> metric = 0.0 | repartition = [10, 0]
|   |   |   |   [17] feature 1 > 148.0 -> metric = 0.0 | repartition = [7, 3]
|   |   |   |   |    Δ metric = +0.6286991436977203
|   |   |   |   |   [34] feature 1 <= 149.0 -> metric = 0.0 | repartition = [0, 1]
|   |   |   |   |   [35] feature 1 > 149.0 -> metric = 0.0 | repartition = [7, 2]
|   |   |   [9] feature 6 > 0.0 -> metric = 0.0 | repartition = [13, 59]
|   |   |   |    Δ metric = +0.5645149157809558
|   |   |   |   [18] feature 1 <= 122.0 -> metric = 0.0 | repartit